Project 1 - Terrence Cummings
- Crime, Schools, and Home Values in Minneapolis

Step 1 - Housing data
Read in housing data from "Open Minneapolis" website using APIs
Store in dataframe

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from datetime import datetime



In [12]:
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Property_Sales_2015_to_2019/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

msp_housing_data = requests.get(url).json()

